In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Load the data
df1 = pd.read_csv("muaBanDat.csv")
df1.head()

,price,address,price_m2,direction,property_legal_document,land_type,pty_characteristics,size,width,length
0,"2,95 tỷ","Đường Võ Văn Kiệt, Phường An Lạc, Quận Bình Tâ...","65,56 triệu/m²",NaN,Đã có sổ,Đất thổ cư,Thổ cư toàn bộ,45 m²,4 m,11.25 m
1,"21,50 tỷ","Đường số 49, Phường Tân Tạo, Quận Bình Tân, Tp...","107,50 triệu/m²",Bắc,Đã có sổ,Đất thổ cư,NaN,200 m²,10 m,20 m
2,"3,65 tỷ","Đường Thới An 10, Phường Thới An, Quận 12, Tp ...","57,03 triệu/m²",Bắc,Đã có sổ,Đất thổ cư,Mặt tiền,64 m²,4 m,16 m
3,630 triệu,"., Xã Tân An Hội, Huyện Củ Chi, Tp Hồ Chí Minh","7,88 triệu/m²",NaN,Đã có sổ,Đất thổ cư,"Mặt tiền,Nở hậu,Hẻm xe hơi,Thổ cư toàn bộ",80 m²,5 m,16 m
4,350 triệu,"Đường Nguyễn Thị Lắng, Xã Tân Phú Trung, Huyện...","4,38 triệu/m²",NaN,Đã có sổ,Đất thổ cư,"Mặt tiền,Nở hậu,Hẻm xe hơi,Thổ cư toàn bộ",80 m²,5 m,16 m


In [2]:
def convert_price(price_str):
    if not isinstance(price_str, str):
        # Handle cases where the value is not a string (e.g., NaN or numeric values)
        return np.nan
    try:
        if 'tỷ' in price_str:
            return float(price_str.replace(' tỷ', '').replace(',', '.')) * 1e9
        elif 'triệu' in price_str:
            return float(price_str.replace(' triệu', '').replace(',', '.')) * 1e6
        elif 'nghìn' in price_str:
            return float(price_str.replace(' nghìn', '').replace(',', '.')) * 1e3
        else:
            # Handle cases where the value is already numeric
            return float(price_str.replace(',', '.'))
    except ValueError:
        print(f"ValueError: Could not convert {price_str} to float.")
        return np.nan

In [3]:
import re
# Function to clean and convert room and toilet columns
def clean_rooms_toilets(value):
    if pd.isna(value):
        return 0
    value = str(value).lower()
    # Extract number if the string contains 'nhiều hơn'
    if 'nhiều hơn' in value:
        match = re.search(r'\d+', value)
        if match:
            return int(match.group(0))
    # Clean and convert the value
    return int(value.replace('phòng', '').strip())

In [4]:
# Load the data
df1 = pd.read_csv("muaBanDat.csv")
df1.head()

df1['title'] = df1['title'].fillna('a')
df1['urgent'] = df1['title'].apply(lambda x: 'yes' if 'gấp' in x.lower() or 'ngộp' in x.lower() else 'no')

# Handle 'location' column if it exists
if 'address' in df1.columns:
    # Xóa các giá trị NaN trong cột 'address'
    df1 = df1.dropna(subset=['address'])

    # Xóa các giá trị kiểu float hoặc không phải chuỗi trong cột 'address'
    df1 = df1[df1['address'].apply(lambda x: isinstance(x, str))]

    # Filter out rows where 'address' contains 'Cập nhật'
    df1 = df1[~df1['address'].str.contains('Cập nhật', case=False, na=False)]

    # Split the 'address' into multiple parts based on ', ' and take the last two columns (district and province)
    df1[['ward','district', 'province']] = df1['address'].apply(lambda x: pd.Series(x.split(', ')[-3:]))

    # Remove "Tp " from the beginning of the 'province' column
    df1['province'] = df1['province'].str.replace(r'^Tp\s+', '', regex=True)

    # Drop the 'address' column after splitting
    df1.drop(columns=['address'], inplace=True)

df1 = df1[df1['province'] == "Hồ Chí Minh"]

# Drop rows where price is "Giá thỏa thuận"
df1 = df1[df1['price'] != "Giá thỏa thuận"]

df1['pty_characteristics'] = df1['pty_characteristics'].fillna('')
df1['land_type'] = df1['land_type'].fillna('')
df1['province'] = df1['province'].fillna('')

# Apply conversion functions
df1['price'] = df1['price'].astype(str).apply(convert_price)

# Ensure all values in 'price_m2' are strings before extracting the numeric part
df1['price_m2'] = df1['price_m2'].astype(str)

# Extract numeric value from 'price_m2', convert commas to periods, and convert to float
df1['price_m2'] = df1['price_m2'].str.extract(r'([\d,.]+)').replace(',', '.', regex=True).astype(float)

# Convert 'price' and 'price_m2' to numeric, if not already
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
df1['price_m2'] = pd.to_numeric(df1['price_m2'], errors='coerce')

# Ensure all values in 'size' are strings before removing 'm²' and converting to float
df1['size'] = df1['size'].astype(str)

# Remove 'm²' and any spaces from 'size', then convert to float
df1['size'] = df1['size'].str.replace(r'm²', '', regex=True).str.strip().astype(float)

# Nếu cột 'size' có giá trị NaN, tính toán lại giá trị cho nó
df1['size'] = df1['size'].fillna(df1['price'] / (df1['price_m2'] * 1000000))

# Làm tròn cột 'size' đến 2 chữ số thập phân
df1['size'] = df1['size'].round(2)

df1.drop(columns=['price_m2'], inplace=True)
df1.drop(columns=['property_legal_document'], inplace=True)
df1.drop(columns=['direction'], inplace=True)
df1.drop(columns=['province'], inplace=True)
df1.drop(columns=['width'], inplace=True)
df1.drop(columns=['length'], inplace=True)
df1.drop(columns=['title'], inplace=True)


# Display the updated DataFrame
df1.head()


ValueError: Could not convert 780.000 đ to float.


,price,land_type,pty_characteristics,size,urgent,ward,district
0,2950000000.00,Đất thổ cư,Thổ cư toàn bộ,45.00,no,Phường An Lạc,Quận Bình Tân
1,21500000000.00,Đất thổ cư,,200.00,no,Phường Tân Tạo,Quận Bình Tân
2,3650000000.00,Đất thổ cư,Mặt tiền,64.00,no,Phường Thới An,Quận 12
3,630000000.00,Đất thổ cư,"Mặt tiền,Nở hậu,Hẻm xe hơi,Thổ cư toàn bộ",80.00,no,Xã Tân An Hội,Huyện Củ Chi
4,350000000.00,Đất thổ cư,"Mặt tiền,Nở hậu,Hẻm xe hơi,Thổ cư toàn bộ",80.00,no,Xã Tân Phú Trung,Huyện Củ Chi


In [10]:
# Kiểm tra giá trị thiếu
print(df1.isnull().sum())
print(df1.shape)

price                  1
land_type              0
pty_characteristics    0
size                   0
ward                   0
district               0
dtype: int64
(3043, 6)


In [5]:
# Convert 'price' and 'size' columns to numeric, forcing errors to NaN
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
df1['size'] = pd.to_numeric(df1['size'], errors='coerce')

# Drop rows where 'price' or 'size' are NaN
df1 = df1.dropna(subset=['price', 'size'])

# Compute IQR for price and size
Q1 = df1[['price', 'size']].quantile(0.25)
Q3 = df1[['price', 'size']].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers
outliers = (df1[['price', 'size']] < (Q1 - 1.5 * IQR)) | (df1[['price', 'size']] > (Q3 + 1.5 * IQR))

# Remove outliers
df1 = df1[~outliers.any(axis=1)]

# Print the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:")
print(df1.shape)


Shape of cleaned DataFrame:
(2488, 7)


In [12]:
print(df1.describe())
df1.shape

               price    size
count        2488.00 2488.00
mean   4547346704.18  110.45
std    3260826476.90   57.73
min      79000000.00    1.20
25%    2400000000.00   70.83
50%    3800000000.00   95.00
75%    6000000000.00  132.10
max   15500000000.00  344.00


(2488, 6)

In [6]:
df1.to_csv('landDataset.csv',encoding="utf-8-sig",index=False)

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

df = pd.read_csv('landDataset.csv')

# Assuming df is already prepared
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'land_type', 'pty_characteristics',"urgent"]])

# Combine encoded features with numeric features
X = np.hstack((encoded_features.toarray(), df[['size']].values))
y = df['price']

# Train the model
model = LinearRegression()
model.fit(X, y)

# Create a DataFrame for the new house
new_house = pd.DataFrame({
'ward': ['Phường Hiệp Phú (Quận 9 cũ)'],
'district': ['Thành phố Thủ Đức'],
'size': [20],
'land_type': ['Đất thổ cư'],
'pty_characteristics': ['Thổ cư toàn bộ'],
'urgent': ['no'],
})

# Encode the new house data
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'land_type', 'pty_characteristics',"urgent"]]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size']].values))

# Dự đoán giá nhà mới bằng mô hình Random Forest
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Linear Regression: {predicted_price[0]:,.0f} VND")

Giá nhà dự đoán bằng Linear Regression: 4,416,126,085 VND


In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor

# Tải dữ liệu
df = pd.read_csv('houseDataset.csv')

# Giả định df đã được chuẩn bị
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell']])

# Kết hợp các đặc trưng đã mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Huấn luyện mô hình Gradient Boosting
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X, y)

# Tạo DataFrame cho nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
'district': ['Quận 5'],
'size': [52],
'rooms': [3],
'toilets': [2],
'floors': [2],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Nội thất đầy đủ'],
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Gradient Boosting
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Gradient Boosting: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Gradient Boosting: 4,913,709,749 VND


In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

# Tải dữ liệu
df = pd.read_csv('houseDataset.csv')

# Giả định df đã được chuẩn bị
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell']])

# Kết hợp các đặc trưng đã mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Huấn luyện mô hình Ridge Regression
model = Ridge(alpha=1.0)  # alpha là tham số điều chỉnh
model.fit(X, y)

# Tạo DataFrame cho nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
'district': ['Quận 5'],
'size': [52],
'rooms': [3],
'toilets': [2],
'floors': [2],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Nội thất đầy đủ'],
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Ridge Regression
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Ridge Regression: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Ridge Regression: 6,848,288,506 VND


In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso

# Tải dữ liệu
df = pd.read_csv('houseDataset.csv')

# Giả định df đã được chuẩn bị
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['ward', 'district', 'house_type', 'furnishing_sell']])

# Kết hợp các đặc trưng đã mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Huấn luyện mô hình Lasso Regression
model = Lasso(alpha=0.1)  # alpha là tham số điều chỉnh
model.fit(X, y)

# Tạo DataFrame cho nhà mới
new_house = pd.DataFrame({
'ward': ['Phường 2'],
'district': ['Quận 5'],
'size': [52],
'rooms': [3],
'toilets': [2],
'floors': [2],
'house_type': ['Nhà ngõ, hẻm'],
'furnishing_sell': ['Nội thất đầy đủ'],
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['ward', 'district', 'house_type', 'furnishing_sell']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá nhà mới bằng mô hình Lasso Regression
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán bằng Lasso Regression: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán bằng Lasso Regression: 6,962,793,704 VND


d:\Workspace python\RealEstatePredict\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.443e+20, tolerance: 2.387e+18
  model = cd_fast.enet_coordinate_descent(
